In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import lfilter

# Parâmetros
num_bits = 1000  # Número de bits
fc = 1000  # Frequência da portadora (Hz)
fs = 10000  # Frequência de amostragem (Hz)
Rb = 100  # Taxa de bits (bits por segundo)
T = 1 / Rb  # Tempo de bit
A = 1  # Amplitude da portadora

# 1. Informação binária
data = np.random.randint(0, 2, num_bits)


# 2. Mapeamento bit para símbolo (QAM)
def bits_to_qam(data):
    symbols = []
    for i in range(0, len(data), 2):
        I = 2 * data[i] - 1
        Q = 2 * data[i + 1] - 1 if i + 1 < len(data) else 0
        symbols.append(complex(I, Q))
    return np.array(symbols)


symbols = bits_to_qam(data)

# 3. Modulação QAM
samples_per_symbol = int(fs / Rb)
time = np.arange(0, len(symbols) * T, 1 / fs)
modulated_signal = np.zeros(len(time), dtype=complex)

for i, symbol in enumerate(symbols):
    modulated_signal[i * samples_per_symbol : (i + 1) * samples_per_symbol] = symbol

# 4I. Tirar parte real
I_signal = modulated_signal.real

# 5I. Aplicar filtro 4PAM
pam4_levels = np.array([-3, -1, 1, 3])  # Níveis do 4PAM
pam4_filter = np.array([1, 1, 1, 1])  # Exemplo de filtro 4PAM com média móvel

I_4pam = np.digitize(I_signal, bins=[-2, 0, 2]) - 2  # Quantização para níveis 4PAM
I_4pam = pam4_levels[I_4pam]
I_filtered = lfilter(pam4_filter, 1, I_4pam)

# 6I. Multiplicar por portadora
carrier_I = A * np.cos(2 * np.pi * fc * time)
I_modulated = I_filtered * carrier_I

# 4Q. Tirar parte imaginária
Q_signal = modulated_signal.imag

# 5Q. Aplicar filtro 4PAM
Q_4pam = np.digitize(Q_signal, bins=[-2, 0, 2]) - 2  # Quantização para níveis 4PAM
Q_4pam = pam4_levels[Q_4pam]
Q_filtered = lfilter(pam4_filter, 1, Q_4pam)

# 6Q. Multiplicar por portadora
carrier_Q = A * np.sin(2 * np.pi * fc * time)
Q_modulated = Q_filtered * carrier_Q

# 7. Soma dos sinais
output_signal = I_modulated - Q_modulated

# Visualização
plt.figure(figsize=(12, 8))

plt.subplot(4, 1, 1)
plt.plot(time, I_signal, label="I_signal")
plt.title("I Signal")
plt.xlabel("Time (s)")
plt.ylabel("Amplitude")
plt.legend()

plt.subplot(4, 1, 2)
plt.plot(time, Q_signal, label="Q_signal", color="orange")
plt.title("Q Signal")
plt.xlabel("Time (s)")
plt.ylabel("Amplitude")
plt.legend()

plt.subplot(4, 1, 3)
plt.plot(time, I_modulated, label="I_modulated")
plt.title("I Modulated Signal")
plt.xlabel("Time (s)")
plt.ylabel("Amplitude")
plt.legend()

plt.subplot(4, 1, 4)
plt.plot(time, Q_modulated, label="Q_modulated", color="orange")
plt.title("Q Modulated Signal")
plt.xlabel("Time (s)")
plt.ylabel("Amplitude")
plt.legend()

plt.tight_layout()
plt.show()

plt.figure(figsize=(12, 4))
plt.plot(time, output_signal, label="Output Signal")
plt.title("Output Signal")
plt.xlabel("Time (s)")
plt.ylabel("Amplitude")
plt.legend()
plt.show()

NotImplementedError: input type 'int64' not supported